In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df3 = pd.read_csv('./df3.csv')
df3.head()

<ipython-input-2-3ca6493518c8>:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df3 = pd.read_csv('./df3.csv')


,gender,car,reality,work_phone,phone,email,family_size,begin_month,credit,employed_age,...,occyp_type_Realty agents,occyp_type_Sales staff,occyp_type_Secretaries,occyp_type_Security staff,occyp_type_State servant_unanswered,occyp_type_Student_unanswered,occyp_type_Waiters/barmen staff,occyp_type_Working_unanswered,occyp_type_retired,edu_type_encoded
0,0,0,0,0,0,0,2.0,6.0,1.0,25.0,...,0,0,0,0,0,0,0,0,0,4
1,0,0,1,0,0,1,3.0,5.0,1.0,26.0,...,0,0,0,0,0,0,0,0,0,2
2,1,1,1,0,1,0,2.0,22.0,2.0,40.0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,1,0,1,0,2.0,37.0,0.0,35.0,...,0,1,0,0,0,0,0,0,0,2
4,0,1,1,0,0,0,2.0,26.0,2.0,35.0,...,0,0,0,0,0,0,0,0,0,4


In [18]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36457 entries, 0 to 36456
Data columns (total 53 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   gender                                      36457 non-null  int64  
 1   car                                         36457 non-null  int64  
 2   reality                                     36457 non-null  int64  
 3   work_phone                                  36457 non-null  int64  
 4   phone                                       36457 non-null  int64  
 5   email                                       36457 non-null  int64  
 6   family_size                                 36457 non-null  float64
 7   begin_month                                 36457 non-null  float64
 8   credit                                      36457 non-null  object 
 9   employed_age                                36457 non-null  float64
 10  age       

In [3]:
train = df3[:26457]
train.shape

(26457, 53)

In [4]:
test = df3[26457:]
test.shape

(10000, 53)

In [5]:
train["credit"] = train["credit"].astype(float).astype(int)

<ipython-input-5-cd75d914b685>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["credit"] = train["credit"].astype(float).astype(int)


In [6]:
train_x=train.drop('credit', axis=1)
train_y=train[['credit']]

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(train_x, train_y,  stratify=train_y, test_size=0.25,random_state = 1)

In [29]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, log_loss
from sklearn.metrics import f1_score
#from tensorflow.keras.utils import to_categorical

rfc=RandomForestClassifier()
rfc.fit(X_train, y_train)
y_pred=rfc.predict_proba(X_val)

#print(f"log_loss: {log_loss(to_categorical(y_val['credit']), y_pred)}")

<ipython-input-29-fdf4d2a7177d>:7: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_train, y_train)


In [14]:
y_train

,credit
13791,2.0
14703,1.0
18743,1.0
22394,2.0
2440,1.0
...,...
23720,2.0
1372,2.0
23995,2.0
20294,2.0


In [30]:
test = test.drop(['credit'], axis=1)
test.shape

(10000, 52)

In [32]:
pred = rfc.predict_proba(test)
pred

array([[0.03, 0.21, 0.76],
       [0.5 , 0.22, 0.28],
       [0.  , 0.03, 0.97],
       ...,
       [0.06, 0.13, 0.81],
       [0.02, 0.51, 0.47],
       [0.12, 0.25, 0.63]])

In [33]:
sub = pd.read_csv('./sample_submission.csv')

In [34]:
sub.head()

,index,0,1,2
0,26457,0,0,0
1,26458,0,0,0
2,26459,0,0,0
3,26460,0,0,0
4,26461,0,0,0


In [38]:
sub['0']=pred[:,0]
sub['1']=pred[:,1]
sub['2']=pred[:,2]
sub.head()

,index,0,1,2
0,26457,0.030000,0.210,0.760000
1,26458,0.500000,0.220,0.280000
2,26459,0.000000,0.030,0.970000
3,26460,0.041667,0.060,0.898333
4,26461,0.069167,0.295,0.635833


In [40]:
sub.to_csv('rfcsubmission.csv', index=False) # 점수가 1.06어쩌구인데 가능한건가.... 어이없네

In [44]:
from lightgbm import LGBMClassifier
lgbm_wrapper = LGBMClassifier(n_estimators=300)
evals = [(X_val, y_val)]
lgbm_wrapper.fit(X_train, y_train, eval_metric="logloss", eval_set=evals)
pred1 = lgbm_wrapper.predict_proba(test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005760 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 688
[LightGBM] [Info] Number of data points in the train set: 19842, number of used features: 50
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score -2.105688
[LightGBM] [Info] Start training from score -1.440238
[LightGBM] [Info] Start training from score -0.444154


In [45]:
pred1

array([[0.05097172, 0.0597665 , 0.88926178],
       [0.15430264, 0.23228277, 0.61341459],
       [0.03313943, 0.05340892, 0.91345165],
       ...,
       [0.04860975, 0.09168275, 0.8597075 ],
       [0.04920237, 0.16961744, 0.78118018],
       [0.08285366, 0.15383701, 0.76330933]])

In [46]:
sub['0']=pred1[:,0]
sub['1']=pred1[:,1]
sub['2']=pred1[:,2]
sub.head()

,index,0,1,2
0,26457,0.050972,0.059766,0.889262
1,26458,0.154303,0.232283,0.613415
2,26459,0.033139,0.053409,0.913452
3,26460,0.094821,0.075189,0.829990
4,26461,0.058136,0.199280,0.742583


In [47]:
sub.to_csv('./lgbm.csv', index=False) # 0.75어쩌구로 rf보다 훨씬 결과 좋음

# lgbm 파라미터 튜닝

In [8]:
def lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha):
  params = {
      "n_estimators" : 500, "learning_rate": 0.02,
      'max_depth': int(round(max_depth)),
      'num_leaves': int(round(num_leaves)),
      'min_child_samples': int(round(min_child_samples)),
      'min_child_weight': int(round(min_child_weight)),
      'subsample': max(min(subsample, 1), 0), # 0 과 1 사이
      'colsample_bytree': max(min(colsample_bytree, 1), 0),
      'max_bin':  max(int(round(max_bin)),10),
      'reg_lambda': max(reg_lambda,0),
      'reg_alpha': max(reg_alpha, 0)

  }
  lgb_model = LGBMClassifier(**params)
  lgb_model.fit(X_train, y_train, eval_set=[(X_train, train_y), (X_val, y_val)], eval_metric= 'auc', verbose= 100,
                early_stopping_rounds= 100)
  valid_proba = lgb_model.predict_proba(test)[:, 1]


  return valid_proba

In [9]:
lgb_roc_eval

<function __main__.lgb_roc_eval(max_depth, num_leaves, min_child_samples, min_child_weight, subsample, colsample_bytree, max_bin, reg_lambda, reg_alpha)>

In [53]:
lgbm_params = {'n_estimators':[200,300,400],
               'max_depth':[3,5,7],
               'subsample':[0.5, 1]}
lgbm_tune = LGBMClassifier(lgbm_params)
evals = [(X_val, y_val)]
from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(lgbm_tune, param_distributions= lgbm_params , n_iter= 30  , random_state= 100 , n_jobs= -1)


In [ ]:
rs.fit(X_train , y_train)

In [ ]:
rs.best_params

In [ ]:
pred = rs.predict_proba(test)
pred